In [1]:
import torch
import os
from ray import tune
from ray.tune import Analysis

ImportError: cannot import name 'Analysis' from 'ray.tune' (c:\Code\Master\hydro-ml\.venv\lib\site-packages\ray\tune\__init__.py)

In [8]:
from ray.tune.analysis import ExperimentAnalysis

# Set the experiment folder path
exp_folder_path = "../ray_results/inflow_forecasting/timestamp"



# Load the trial information
#analysis = ExperimentAnalysis(trials=)
